# TFT Simulation

In [1]:
from lib import *
from params_bom_hardware_components import *
from token_creator import *
simulation = j.tools.tfgrid_simulator.default
simulation.token_creator.tft_burn = tft_burn
simulation.token_creator.tft_cultivate = tft_cultivate
simulation.token_creator.tft_farm = tft_farm
simulation.token_creator.difficulty_level_get = difficulty_level_get
from params_environment import *

In [2]:
#PRIMARY PARAMETERS

# month:growth_percent of nodes being added
simulation.growth_percent_set("3:5,11:8,24:10,36:12,48:10,60:10,61:0")
simulation.difficulty_level_set("0:2,71:2")
simulation.nodesbatch_start_set(nrnodes=1500, months_left=36, tft_farmed_before_simulation=20 * 1000 * 1000)

# do the calculation of the simulation
simulation.calc(nrnodes_new="0:5,6:150,12:1000,13:0")


In [3]:
md(f"""
## Simulation Arguments 

- nr of nodes at start: {simulation.nodebatches[0].nrnodes}
- nr of months left (avg out) for farming of startingnodes: {simulation.nodebatches[0].months_left}
- nr of TFT already farmed: { j.data.types.numeric.clean(simulation.nodebatches[0].tft_farmed_before_simulation)} TFT


""")


## Simulation Arguments 

- nr of nodes at start: 1500
- nr of months left (avg out) for farming of startingnodes: 36
- nr of TFT already farmed: 20,000,000 TFT




## Device Details

In [4]:

md(f"""
### Device Details

- nr of TB netto usable = {round(device_edge.su,1)} TB
- nr of cpu cores = {round(device_edge.cru,1)} Cores
- power used = {device_edge.power} watt
- performance in passmark = {round(device_edge.cu_passmark*device_edge.cu)}

""")


### Device Details

- nr of TB netto usable = 58.3 TB
- nr of cpu cores = 16.0 Cores
- power used = 222 watt
- performance in passmark = 24479



## Growth per month

In [68]:
x=np.arange(60)
fig = go.Figure(data=go.Scatter(x=x, y= simulation.sheet.rows.growth_percent.cells[0:60]))
fig.update_layout(title = 'Growth Percent Per Month',showlegend=False)
fig.show()

cells= simulation.sheet.rows.nrnodes_new.cells[0:60]
fig = go.Figure(data=go.Scatter(x=x, y=cells))
fig.update_layout(title = 'New Nr Nodes per Month',showlegend=False)
fig.show()


# plt.figure(title='New Nr Nodes per Month' )
# cells= simulation.sheet.rows.nrnodes_new.cells[0:60]
# plt.plot(np.arange(60), cells)
# plt.show()

# plt.figure(title='Nr Nodes per Month (per 1000)' )
# cells=[i/1000 for i in simulation.sheet.rows.nrnodes.cells[0:60]]
# plt.plot(np.arange(60), cells)
# plt.show()



### Number of racks per month


In [6]:
plt.figure(title='New Racks Occupied per Month' )
cells= [i/44 for i in simulation.sheet.rows.rackspace_u.cells[0:60]]
plt.plot(np.arange(60), cells)
plt.show()

plt.figure(title='Per per kwatt/h per Month used.' )
cells= [i/44 for i in simulation.sheet.rows.power_kw.cells[0:60]]
plt.plot(np.arange(60), cells)
plt.show()

## TFT Growth

In [7]:
plt.figure(title='TFT Farmed, Cultivated, Sold, Burned (per 1000) for first 29 months (1 on graph is month 2)')
names=["farmed","cultivated","sold","burned","total"]
colors=["orange","green","blue","yellow","red"]
pos=[1,1,-1,-1,1]
startmonth=0
for i in range(5):
    name=names[i]
    color=colors[i]
    cells=[]
    for x in range(1,30):
        y=simulation.sheet.rows["tft_%s"%name].cells[x]*pos[i]
        cells.append(y/1000)   
    line=plt.plot([i for i in range(1,30)],cells,colors=[color])
    xx=15
    yy=300
    label = plt.label([name], x=[2],y=[yy*20+(i+1)*7*yy], colors=[color])
plt.show()
plt.figure(title='TFT Farmed, Cultivated, Sold, Burned (per 1000) for last 30 months (1 on graph is month 30)')
names=["farmed","cultivated","sold","burned","total"]
colors=["orange","green","blue","yellow","red"]
pos=[1,1,-1,-1,1]
startmonth=30
for i in range(5):
    name=names[i]
    color=colors[i]
    cells=[]
    for x in range(startmonth,60):
        y=simulation.sheet.rows["tft_%s"%name].cells[x]*pos[i]
        cells.append(y/1000)    
    line=plt.plot([i for i in range(1,30)],cells,colors=[color])
    xx=15
    yy=4000
#     label = plt.label([name], x=[2],y=[yy*20+(i+1)*7*yy], colors=[color])

plt.show()

## Cumulative TOKENS CREATED / BURNED / CULTIVATED / SOLD (in millions)


In [65]:
m=1000000
rf=[i/m for i in simulation.rows.tft_farmed.aggregate("Y",aggregate_type="SUM")]
rb=[i/m for i in simulation.rows.tft_burned.aggregate("Y",aggregate_type="SUM")]
rs=[i/m for i in simulation.rows.tft_sold.aggregate("Y",aggregate_type="SUM")]
rc=[i/m for i in simulation.rows.tft_cultivated.aggregate("Y",aggregate_type="SUM")]

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])

labels=["tft_farmed","tft_burned","tft_sold","tft_cultivated"]
for year in range(2):
    data=[rf[year],rb[year],rs[year],rc[year]]
    graph = go.Pie(labels=labels, values=data, name="Year %s"%(year+1))
    fig.add_trace(graph,1,year+1)
fig.show()

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
y=1
for year in range(2,4):
    data=[rf[year],rb[year],rs[year],rc[year]]
    graph = go.Pie(labels=labels, values=data, name="Year %s"%(year+1))
    fig.add_trace(graph,1,y)
    y+=1
fig.show()
# md("### Details")



In [45]:


import plotly.graph_objects as go
fig = go.Figure()
# fig.add_trace(go.Scatter(y=[2, 1, 4, 3]))
x=[1,2,3,4,5]
fig.add_trace(go.Bar(x=x,y=rf[0:5],name="Farming"))  #,text=["s sds s"]
fig.add_trace(go.Bar(x=x,y=rc[0:5],name="Cultivating"))
fig.add_trace(go.Bar(x=x,y=rs[0:5],name="Sold"))
fig.add_trace(go.Bar(x=x,y=rb[0:5],name="Burned"))
fig.update_layout(title = 'Farming / Cultivation / Burning Comparison',showlegend=True)
fig.show()

In [35]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(y=[2, 1, 4, 3]))
fig.add_trace(go.Bar(y=[1, 4, 3, 2]))
fig.update_layout(title = 'Hello Figure')
fig.show()

### Cost/Revenue per month

### Revenu per year / Revenue per year extrapolated for 5 years

In [ ]:
import matplotlib.pyplot as plt4
fig1, axs = plt4.subplots(1,2)
plt2.rcParams['figure.figsize'] = [18, 10]
a1=axs[0]
a2=axs[1]
b1=a1.bar(['Y1', 'Y2', 'Y3', 'Y4', 'Y5'], [int(i/1000000) for i in simulation.rows.revenue.aggregate("Y",aggregate_type="SUM")][0:5])
# a1.title("Cultivation Revenue Per Year (million USD).")
b2=a2.bar(['Y1', 'Y2', 'Y3', 'Y4', 'Y5'], [int(i) for i in simulation.rows.revenue_extrapolated.aggregate("Y",aggregate_type="LAST")][0:5])
# a2.title("Cultivation Revenue Per Year (million USD).")
plt2.show()
# print(simulation.rows.revenue_extrapolated)

### Token price per month


In [ ]:
plt.figure(title='Token Price (TFT).')
cells=simulation.rows.tokenprice.cells[0:60]
plt.plot(np.arange(1,len(cells)), [round(i,2) for i in cells])
plt.show()